In [2]:
import pandas as pd
from scipy.optimize import linprog
import numpy as np
from gurobipy import GRB
import gurobipy as gb
from gurobipy import Model, GRB, quicksum


ModuleNotFoundError: No module named 'gurobipy'

In [ ]:
model = gb.Model("Optimization Problem")

In [ ]:
# Define variables
# Number of production facilities, transshipment hubs, and refinement centers
num_prod_facilities = len(data['capacity_direct_prod_facilities']) + len(data['capacity_transship_prod_facilities'])
num_transship_hubs = len(data['capacity_transship_distribution_centers'])
num_refinement_centers = len(data['refinement_demand'])

# Decision variables for direct routes from production facilities to refinement centers
x_direct = model.addVars(num_prod_facilities, num_refinement_centers, lb=0, vtype=GRB.CONTINUOUS, name="Direct_Transport")

# Decision variables for routes from production facilities to transshipment hubs
x_prod_to_transship = model.addVars(num_prod_facilities - 25, num_transship_hubs, lb=0, vtype=GRB.CONTINUOUS, name="Prod_to_Transship")

# Decision variables for routes from transshipment hubs to refinement centers
x_transship_to_refinement = model.addVars(num_transship_hubs, num_refinement_centers, lb=0, vtype=GRB.CONTINUOUS, name="Transship_to_Refinement")

# Objective function: Minimize the total transportation cost
model.setObjective(
    quicksum(data['cost_prod_to_refinement'].loc[(data['cost_prod_to_refinement']['ProductionFacility'] == i+1) & 
                                                 (data['cost_prod_to_refinement']['RefinementCenter'] == j+1), 'Cost'].values[0] * x_direct[i,j] 
             for i in range(25) for j in range(num_refinement_centers)) +
    quicksum(data['cost_prod_to_transshipment'].loc[(data['cost_prod_to_transshipment']['ProductionFacility'] == i+1) & 
                                                   (data['cost_prod_to_transshipment']['TransshipmentHub'] == j+1), 'Cost'].values[0] * x_prod_to_transship[i,j] 
             for i in range(15) for j in range(num_transship_hubs)) +
    quicksum(data['cost_transshipment_to_refinement'].loc[(data['cost_transshipment_to_refinement']['TransshipmentHub'] == i+1) & 
                                                         (data['cost_transshipment_to_refinement']['RefinementCenter'] == j+1), 'Cost'].values[0] * x_transship_to_refinement[i,j] 
             for i in range(num_transship_hubs) for j in range(num_refinement_centers)), 
    GRB.MINIMIZE)

# Supply constraints for direct production facilities
for i in range(25):
    model.addConstr(quicksum(x_direct[i,j] for j in range(num_refinement_centers)) <= data['capacity_direct_prod_facilities'].loc[i, 'Capacity'], 
                    name=f"Supply_Constraint_Direct_{i}")

# Supply constraints for production facilities requiring transshipment
for i in range(15):
    model.addConstr(quicksum(x_prod_to_transship[i,j] for j in range(num_transship_hubs)) <= data['capacity_transship_prod_facilities'].loc[i, 'Capacity'], 
                    name=f"Supply_Constraint_Transship_{i}")

# Capacity constraints for transshipment hubs
for j in range(num_transship_hubs):
    model.addConstr(quicksum(x_prod_to_transship[i,j] for i in range(15)) <= data['capacity_transship_distribution_centers'].loc[j, 'Capacity'], 
                    name=f"Transship_Capacity_{j}")

# Demand constraints for refinement centers
for j in range(num_refinement_centers):
    model.addConstr(quicksum(x_direct[i,j] for i in range(25)) + quicksum(x_transship_to_refinement[k,j] for k in range(num_transship_hubs)) == data['refinement_demand'].loc[j, 'Demand'], 
                    name=f"Demand_Constraint_{j}")

# Solve the model
model.optimize()

# Results
if model.status == GRB.OPTIMAL:
    total_cost = model.objVal
    print("Optimal Total Transportation Cost:", total_cost)
else:
    print("Optimization was not successful.")


In [ ]:
# A list of list of costs
costs = [[21, 50, 40], [35, 30, 22], [55, 20, 25]]
demand = [200000, 600000, 225000]
supply = [275000, 400000, 300000]

# Create the a single class of decision variables
x = model.addVars(3, 3, lb=0, vtype=GRB.CONTINUOUS, name="Transportation Plan")

# The objective function
model.setObjective(gb.quicksum(costs[i][j]*x[i,j] for i in range(3) for j in range(3)), GRB.MINIMIZE)

# Add the supply constraints
for i in range(3):
    model.addConstr(gb.quicksum(x[i,j] for j in range(3)) == supply[i], name="Supply Constraint %i" %i)

# Add the demand constraints
for j in range(3):
    model.addConstr(gb.quicksum(x[i,j] for i in range(3)) <= demand[j], name="Demand Constraint %i" %j)

# Optimally solve the problem
model.optimize()

# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

# Value of the objective function
print("Total Transportation cost: ", model.objVal)

# Print the decision variables
print(model.printAttr('X'))

In [ ]:
# A list of list of costs
source_costs = [[11, 10, 26, 29], [9, 12, 27, 26]]
trans_costs = [[12, 16], [13, 15]]

# Create the a single class of decision variables where
# From = {𝑫𝒂,𝑯𝒐} and To = {𝑪𝒉,𝑳𝑨,𝑺𝑭,𝑵𝒀}.
x = model.addVars(2, 4, lb=0, vtype=GRB.CONTINUOUS, name="Source Nodes")
# From = {𝑪𝒉,𝑳𝑨} and To = {𝑺𝑭,𝑵𝒀}.
y = model.addVars(2, 2, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment Nodes")

# The objective function
source_objective = gb.quicksum(source_costs[i][j]*x[i,j] for i in range(2) for j in range(4))
trans_objective = gb.quicksum(trans_costs[i][j]*y[i,j] for i in range(2) for j in range(2))
model.setObjective(source_objective + trans_objective, GRB.MINIMIZE)

# Add the supply constraints from source nodes
model.addConstr(gb.quicksum(x[0,j] for j in range(4)) <= 200, name="Supply Constraint 1")
model.addConstr(gb.quicksum(x[1,j] for j in range(4)) <= 160, name="Supply Constraint 2")
    
# Add the supply constraints from transshipment nodes
model.addConstr(gb.quicksum(x[i,0] for i in range(2)) <= 90, name="Transship Capacity 1")
model.addConstr(gb.quicksum(x[i,1] for i in range(2)) <= 80, name="Transship Capacity 2")

# Add the flow balance constrainits
model.addConstr(gb.quicksum(x[i,0] for i in range(2)) == gb.quicksum(y[0,k] for k in range(2)), name="Flow Balance 1")
model.addConstr(gb.quicksum(x[i,1] for i in range(2)) == gb.quicksum(y[1,k] for k in range(2)), name="Flow Balance 2")

# Add the demand constraints
model.addConstr(gb.quicksum(x[i,2] + y[i,0] for i in range(2)) == 140, name="Demand Constraint 1")
model.addConstr(gb.quicksum(x[i,3] + y[i,1] for i in range(2)) == 140, name="Demand Constraint 2")

# Ratio constraint
model.addConstr(0.6*gb.quicksum(y[i,j] for i in range(2) for j in range(2)) <= 0.4*gb.quicksum(x[i,j] for i in range(2) for j in range(2,4)), name="Ratio constraint")

# Optimally solve the problem
model.optimize()

# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

# Value of the objective function
print("Total Transportation cost: ", model.objVal)

# Print the decision variables
print(model.printAttr('X'))

In [2]:
df_direct_production_capacity = pd.read_csv('/Users/youziya/Downloads/Capacity_for_Direct_Production_Facilities.csv')
df_transship_production_capacity = pd.read_csv('/Users/youziya/Downloads/Capacity_for_Transship_Production_Facilities.csv')
df_transship_distribution_capacity = pd.read_csv('/Users/youziya/Downloads/Capacity_for_Transship_Distribution_Centers.csv')
df_cost_production_to_refinement = pd.read_csv('/Users/youziya/Downloads/Cost_Production_to_Transshipment.csv')
df_cost_production_to_transshipment = pd.read_csv('/Users/youziya/Downloads/Cost_Production_to_Refinement.csv')
df_cost_transshipment_to_refinement = pd.read_csv('/Users/youziya/Downloads/Cost_Transshipment_to_Refinement.csv')
df_refinement_demand = pd.read_csv('/Users/youziya/Downloads/Refinement_Demand.csv')


In [3]:
dfs = {
    'Direct Production Capacity': df_direct_production_capacity,
    'Transship Production Capacity': df_transship_production_capacity,
    'Transship Distribution Capacity': df_transship_distribution_capacity,
    'Cost Production to Refinement': df_cost_production_to_refinement,
    'Cost Production to Transshipment': df_cost_production_to_transshipment,
    'Cost Transshipment to Refinement': df_cost_transshipment_to_refinement,
    'Refinement Demand': df_refinement_demand
}

dfs


{'Direct Production Capacity':     ProductionFacility  Capacity
 0                    1       462
 1                    2       103
 2                    3       460
 3                    4       325
 4                    5       227
 5                    6       217
 6                    7       205
 7                    8       521
 8                    9       548
 9                   10       191
 10                  11       361
 11                  12       411
 12                  13       104
 13                  14       155
 14                  15       285
 15                  16       109
 16                  17       422
 17                  18       438
 18                  19       501
 19                  20       139
 20                  21       462
 21                  22       504
 22                  23       106
 23                  24       132
 24                  25       298,
 'Transship Production Capacity':     ProductionFacility  Capacity
 0                

In [7]:
from scipy.optimize import linprog

# Number of decision variables for each part of the supply chain
num_direct_shipments = n_direct_prod * n_refinement
num_prod_to_trans = n_transship_prod * n_transship_hub
num_trans_to_ref = n_transship_hub * n_refinement

# Constraint matrix and bounds setup
# Initialize with zero arrays
A_eq = np.zeros((n_direct_prod + n_transship_prod + n_transship_hub + n_refinement, total_variables))
b_eq = np.zeros(n_direct_prod + n_transship_prod + n_transship_hub + n_refinement)

# Setting up capacity constraints for direct production facilities
for i in range(n_direct_prod):
    A_eq[i, i*n_refinement:(i+1)*n_refinement] = 1
b_eq[:n_direct_prod] = cap_direct_production

# Setting up capacity constraints for transshipment production facilities
offset = num_direct_shipments
for i in range(n_transship_prod):
    A_eq[n_direct_prod + i, offset + i*n_transship_hub:offset + (i+1)*n_transship_hub] = 1
b_eq[n_direct_prod:n_direct_prod + n_transship_prod] = cap_transship_production

# Setting up capacity constraints for transshipment hubs
offset = num_direct_shipments + num_prod_to_trans
for i in range(n_transship_hub):
    A_eq[n_direct_prod + n_transship_prod + i, offset + i*n_refinement:offset + (i+1)*n_refinement] = 1
b_eq[n_direct_prod + n_transship_prod:n_direct_prod + n_transship_prod + n_transship_hub] = cap_transship_distribution

# Setting up demand constraints at refinement centers
for i in range(n_refinement):
    A_eq[-n_refinement + i, i::n_refinement] = 1  # Direct shipments
    A_eq[-n_refinement + i, offset + i::n_refinement] = 1  # Transshipments
b_eq[-n_refinement:] = demand_refinement

# Solving the linear program
result = linprog(cost_vector, A_eq=A_eq, b_eq=b_eq, method='highs')

# Checking if the solution is successful and obtaining the optimal cost
if result.success:
    optimal_cost = result.fun
else:
    optimal_cost = "Solution unsuccessful"

optimal_cost


NameError: name 'n_direct_prod' is not defined